In [51]:
workspace()

In [52]:
type Variable
    name::AbstractString
    domain::Array
    evidence_index::Int32
    assignment_index::Int32
    
    add_values::Function
    value_index::Function
    domain_size::Function
    set_assignment::Function
    get_assignment_index::Function
    get_evidence::Function
    
    function Variable(name="", domain=[])
        this = new()
        
        this.name = name
        this.domain = copy(domain)
        
        this.evidence_index = 1
        this.assignment_index = 1
        
#         Проверено
        this.add_values = function(values::Array)
            this.domain = vcat(this.domain, value)
        end
        
#       Проверено
        this.value_index = function(value::Any)
            return findfirst(this.domain, value)
        end
        
#         Проверено
        this.domain_size =  function()
            return size(this.domain)[1]
        end
        
#         Проверено
        this.set_assignment = function(value)
            this.assignment_index = this.value_index(value)
        end
        
#         Проверно
        this.get_assignment_index = function()
           return this.assignment_index 
        end
        
        
        this.get_evidence = function()
            return this.domain[this.evidence_index]
        end
        return this
    end
    
end

In [53]:
type Factor
    name::AbstractString
    scope::Array
    size::Int64
    values::Array
    
    get_value::Function
    get_scope::Function
    add_scope::Function
    add_values::Function
    add_value_at_current_assignments::Function
    get_value_at_current_assignments::Function
    
    function Factor(name="NEW", scope=[])
        this = new()
        
        this.name = name
        this.scope = scope
        
        size = 1
        for v in scope
            size *= v.domain_size()
        end
        this.values = zeros(size)
        
        this.size = size
        
#         Проверено
        this.get_scope = function ()
            return this.scope
        end
        
        
        this.add_scope = function (added::Array)
            this.scope = vcat(this.scope, added)
        end
        
#         this.add = function (value::Node)
#             this.value = value
#         end
        
        
#        fixed bag with indexes
        this.add_values =  function(values::Array)
            for val in values
               idx = 0
                for v in this.scope
                    idx = idx * v.domain_size() + v.value_index(val[1]) -1 
                    print(idx)
                    val = val[2:end]
                end
                idx += 1
                this.values[idx] = val[1]
            end
        end
        
        this.get_value = function(variable_values)
           idx = 0
            print(this.scope)
            for v in this.scope
                idx = idx * v.domain_size() + v.value_index(variable_values[1]) - 1
                variable_values = variable_values[2:end]
            end
            idx += 1
            return this.values[idx]
        end
        
#       As I checked - it is work
        this.add_value_at_current_assignments = function(number)
            idx = 0
            
            for v in this.scope
               idx = idx * v.domain_size() + v.get_assignment_index() -1 
            end
            idx += 1
            this.values[idx] =  number
        end
        
        
        this.get_value_at_current_assignments = function()
            
            idx = 0
            
            for v in this.scope
               idx = idx * v.domain_size() + v.get_assignment_index() - 1
            end
            idx += 1
            return this.values[idx]
        end
        
        
        return this
    end
end

In [ ]:
type BN
    name::AbstractString
    Variables::Array{Variables}
end

In [54]:
function restrict_factor(F, var, value)
    newFactorName = F.name * "_{R:}" * var.name * " = " * string(value) * "}"
    newFactorScope = copy(F.get_scope())
    
    var.set_assignment(value)
    deleteat!(newFactorScope, findin(newFactorScope, [var]))
    
    newFactor = Factor(newFactorName, newFactorScope)
    
    rec_restrict_factor(F, newFactorScope, newFactor)
    
    return newFactor
end

restrict_factor (generic function with 1 method)

In [55]:
function rec_restrict_factor(oldFactor, newFactorScope, newFactor)
    if isempty(newFactorScope)
        old_assign = oldFactor.get_value_at_current_assignments()
        newFactor.add_value_at_current_assignments(old_assign)
    else
        print(newFactorScope)
        for var_val in newFactorScope[1].domain
            newFactorScope[1].set_assignment(var_val)
            rec_restrict_factor(oldFactor, newFactorScope[2:end], newFactor)
        end
    end
end

rec_restrict_factor (generic function with 1 method)

In [56]:
function step1(NetFactors, EvidenceVars)
    for i in range(1, size(NetFactors)[1])
        for e_v in EvidenceVars
            if e_v in NetFactors[i].get_scope()
                newFactor = restrict_factor(NetFactors[i], e_v, e_v.get_evidence())
                
                NetFactors[i] = copy(newFactor)
            end
        end
    end
end

step1 (generic function with 1 method)

In [57]:
function multiply_factors(Factors)
   newFactorName = ""
    for i in range(1, size(Factors)[1])
        cur_factor = Factors[i]
        if i == 1
            newFactorName *= cur_factor.name
            continue
        else
            newFactorName = newFactorName * "_" * cur_factor.name * "_"
            continue
        end
        
    end
    
    newFactorScope = Variable[]
    
    for factor in Factors
        for var in factor.get_scope()
            if var in newFactorScope
                continue
            end
            push!(newFactorScope, var)
        end
    end
    
    newFactor = Factor(newFactorName, newFactorScope)
    println("SOS")
    println(newFactorScope)
    rec_multiply_factors(Factors, newFactorScope, newFactor, 1)
    println("PISOS")
    
    return newFactor
    
end

multiply_factors (generic function with 1 method)

In [58]:
function rec_multiply_factors(Factors, newFactorScope, newFactor, total=1)
    println("PI")
    println(newFactorScope)
    if !isempty(newFactorScope)
        print("L")
        for factor in Factors
            print("E")
            total *= factor.get_value_at_current_assignments()
            print("W")
        end

        newFactor.add_value_at_current_assignments(total)
    else
        for var_val in newFactorScope[1].domain()
            newFactorScope[1].set_assignment(var_val)
            print("POPA")
            rec_multiply_factors(Factors, newFactorScope[2:end], newFactor, total)
        end
    end
end

rec_multiply_factors (generic function with 2 methods)

In [59]:
function remove_var(var, new_scope, scopes)
    new_scopes = []
    for s in scopes
        if !(var in s)
             push!(new_scopes, s)
        end
    end
    
    push!(new_scopes, new_scope)
    
    return new_scopes
end

remove_var (generic function with 1 method)

In [60]:
function min_fill_var(scopes, Vars)
    minv = Vars[1]
    (minfill, min_new_scope) = compute_fill(scopes, Vars[0])
    
    for v in Vars[2:end]
        (fill, new_scope) = compute_fill(scopes, v)
        if fill < minfill
            minv = v
            minfill = fill
            
            min_new_scope = new_scope
        end
    end
    return (minv, min_new_scope)

end

min_fill_var (generic function with 1 method)

In [61]:
function compute_fill(scopes, var)
    union = []
    for s in scopes
        if var in s
            for v in s
                if !(v in union)
                   push!(union, v) 
                end
            end
        end
    end
    if var in union
        deleteat!(union, findin(union, var))
    end
    
    return (size(union)[1], union)
end

compute_fill (generic function with 1 method)

In [62]:
function min_fill_ordering(Factors, QueryVar)
    scopes = []
    for f in Factors
       push!(scopes, f.get_scope())
    end
    
    Vars = []
    
    for s in scopes
        for v in s
            if !(v in Vars) && (v != QueryVar)
                push!(Vars, v)
            end
        end
    end
    
    ordering = []
    
    while !empty(Vars)
        (var, new_scope) = min_fill_var(scopes, Vars)
        push!(ordering, var)
        
        if var in Vars
            deleteat!(Vars, findin(Vars, v))
        end
        
        scopes = remove_var(var, new_scope, scopes)
    end
end

min_fill_ordering (generic function with 1 method)

In [63]:
function sum_out_variable(f, var)
    newFactorName = f.name + "_{S: " + var.name + "}"
    newFactorScope = f.get_scope()
    
    deleteat!(newFactorScope, findin(newFactorScope, var))
    
    newFactor = Factor(newFactor, newFactorScope)
    
    rec_sumout_vars(newFactor, f, newFactorScope, var)
    
    return newFactor
end

sum_out_variable (generic function with 1 method)

In [65]:
function rec_sumout_variable(newFactor, f, newFactorScope, variable, summ=0)
   if !(size(newFactorScope)[1])
        for val in variable.domain()
            variable.set_assignment(val)
            curr = f.get_value_at_current_assignments
            summ += curr
        end
        
        newFactor.add_value_at_current_assignments(summ)
        
    else
        for var_val in newFactorScope[1].domain()
            newFactorScope[1].set_assignment(var_val)
            rec_sumout_variable(newFactor, f, newFactorScope[2:end], variable)
        end
    end
end

rec_sumout_variable (generic function with 2 methods)

In [47]:
e = [1, 2, 3]
size(e)[1]

3

In [40]:
deleteat!(e, findin(e, 3))

2-element Array{Int64,1}:
 1
 2

In [66]:
function step2(NetFactors, QueryVar)
   for v in min_fill_ordering(NetFactors, QueryVar)
        f_v = []
        for factor in NetFactors
            if v in factor.get_scope()
               push!(f_v, factor) 
            end
        end
        
        prod_f = multiply_factors(f_v)
        
        g_j = sum_out_variable(prod_f, v)
        
        for factor in f_v
            deleteat!(NetFactors, findin(NetFactors, factor))
        end
        
        push!(NetFactors, g_j)
    end
end

step2 (generic function with 1 method)

In [70]:
function step3(NetFactors)
    remaining_factors_w_Q = multiply_factors(NetFactors)
    
    normalization_factor = 0
    
    for i in remaining_factors_w_Q.values
        normalization_factor += i
    end
    
    if !(normalization_factor)
        error("ZeroDivision")
    end
    
    list_of_pos = []
    for i in remaining_factors_w_Q.values
        push!(list_of_pos, i/normalization_factor)
    end
    
    return list_of_pos
end

step3 (generic function with 1 method)

In [72]:
function VariableElimination(Net, QueryVar, EvidanceVars)
    NetFactors = Net.factors()
    
    step1(NetFactors, EvidanceVars)
    
    step2(NetFactors, QueryVar)
    
    return step3(NetFactors)
end

VariableElimination (generic function with 1 method)

In [79]:
ew = Array{Any}

Array{Any,N}

In [80]:
push!(ew, Variable())

LoadError: MethodError: `push!` has no method matching push!(::Type{Array{Any,N}}, ::Variable)[0m
Closest candidates are:
  push!(::Any, ::Any, [1m[31m::Any[0m)
  push!(::Any, ::Any, [1m[31m::Any[0m, [1m[31m::Any...[0m)
  push!([1m[31m::Array{Any,1}[0m, ::ANY)
  ...[0m

In [77]:
ew

Array{Variable,N}

LoadError: MethodError: `/` has no method matching /(::Float64, ::Char)[0m
Closest candidates are:
  /(::Float64, [1m[31m::Float64[0m)
  /(::Real, [1m[31m::Complex{T<:Real}[0m)
  /(::Union{Float16,Float32,Float64}, [1m[31m::BigFloat[0m)
  ...[0m